In [1]:
import pandas as pd
import altair as alt
from altair import datum
import numpy as np


In [2]:
data_media = pd.read_csv("all_polutants_average.csv")
data_min= pd.read_csv("all_polutants_min.csv")
data_max =pd.read_csv("all_polutants_max.csv")

In [3]:
data_media.columns = data_media.columns.str.replace(" ", "_")
data_min.columns = data_min.columns.str.replace(" ", "_")
data_max.columns = data_max.columns.str.replace(" ", "_")


In [4]:
data_list = [data_media, data_min, data_max]
polutants_list = ["PM10", "PM2.5", "O3", "NO2", "SO2"]

In [5]:
def apply_descricao(poluente):
    if poluente == 'PM10':
        return 'Partículas de diâmetro igual ou inferior a 10 µm'
    elif poluente == 'PM2.5':
        return 'Partículas de diâmetro igual ou inferior a 2.5 µm'
    elif poluente == 'O3':
        return 'Ozono'
    elif poluente == 'NO2':
        return 'Dióxido de azoto'
    elif poluente == 'SO2':
        return 'Dióxido de enxofre'
    return ''

In [6]:
def apply_level (poluente, nivel_poluicao):
    if (poluente == 'PM10'):
        if (nivel_poluicao <= 20):
            return "Muito Bom"
        elif (nivel_poluicao <=35):
            return "Bom"
        elif (nivel_poluicao <= 50):
            return "Médio"
        elif (nivel_poluicao <= 100):
            return "Fraco"
        elif (nivel_poluicao <=1200):
            return "Mau"
    elif (poluente == "PM2.5"):
        if (nivel_poluicao <= 10):
            return "Muito Bom"
        elif (nivel_poluicao <=20):
            return "Bom"
        elif (nivel_poluicao <= 25):
            return "Médio"
        elif (nivel_poluicao <= 50):
            return "Fraco"
        elif (nivel_poluicao <= 800):
            return "Mau"
    elif (poluente == "NO2"):
        if (nivel_poluicao <= 40):
            return "Muito Bom"
        elif (nivel_poluicao <=100):
            return "Bom"
        elif (nivel_poluicao <= 200):
            return "Médio"
        elif (nivel_poluicao <= 400):
            return "Fraco"
        elif (nivel_poluicao <= 1000):
            return "Mau"
    elif (poluente == "O3"):
        if (nivel_poluicao <= 80):
            return "Muito Bom"
        elif (nivel_poluicao <=100):
            return "Bom"
        elif (nivel_poluicao <= 180):
            return "Médio"
        elif (nivel_poluicao <= 240):
            return "Fraco"
        elif (nivel_poluicao <= 600):
            return "Mau"
    elif (poluente == "SO2"):
        if (nivel_poluicao <= 100):
            return "Muito Bom"
        elif (nivel_poluicao <=200):
            return "Bom"
        elif (nivel_poluicao <= 350):
            return "Médio"
        elif (nivel_poluicao <= 500):
            return "Fraco"
        elif (nivel_poluicao <= 1250):
            return "Mau" 

In [33]:
def transform_and_translate (data_list) :
    data_ = [];
    for data in data_list:
        dataset = data.drop(columns=['Air_Quality_Network','Air_Quality_Network_Name', 'Air_Quality_Station_EoI_Code', 'Sampling_Point_Id','Air_Pollutant_Description','Data_Aggregation_Process_Id', 'Source_Of_Data_Flow', 'Verification', 'City_Code', 'Calculation_Time', 'Country', 'Data_Coverage'])
        dataset = dataset[dataset.Air_Pollutant.isin(polutants_list)]
        if dataset["Data_Aggregation_Process"].str.contains("Annual mean / 1 calendar year").any():
            dataset["link_tabela_EEA"] = "https://discomap.eea.europa.eu/App/AirQualityStatistics/index.html?Country=Portugal&DataAggregationProcess=Annual mean / 1 calendar year"
        elif dataset["Data_Aggregation_Process"].str.contains("1 year day max").any():
            dataset["link_tabela_EEA"] = "https://discomap.eea.europa.eu/App/AirQualityStatistics/index.html?Country=Portugal&DataAggregationProcess=1 year day max"
        else:
            dataset["link_tabela_EEA"] = "https://discomap.eea.europa.eu/App/AirQualityStatistics/index.html?Country=Portugal&DataAggregationProcess=1 year day min"
        dataset.rename(columns={"Country" :"país" , 
                                "Air_Quality_Station_Name": "nome_estacao",
                                "Air_Pollutant" : "poluente",
                                "Data_Aggregation_Process" : "metrica",
                                "Year" : "ano",
                                "Air_Pollution_Level" : "nível_de_poluicao",
                                "Unit_Of_Air_Pollution_Level" : "unidade_de_nivel_de_poluicao",
                                #"Data_Coverage" : "cobertura_dos_dados",
                                "Air_Quality_Station_Type":"tipo_de_estacao",
                                "Air_Quality_Station_Area" : "tipo_de_area",
                                "Longitude" : "longitude",
                                "Latitude" : "latitude",
                                "Altitude" : "altitude",
                                "City" : "cidade",
                                "City_Population" : "populacao_da_cidade",
                                #"Calculation_Time" : "periodo_de_calculo",
                                "Link_to_raw_data_(only_E1a/validated_data_from_AQ_e-Reporting)" : "link_dados_raw"
                               }, inplace=True)
        dataset["metrica"] = dataset.metrica.str.replace("Annual mean / 1 calendar year", "Média anual")
        dataset["metrica"] = dataset.metrica.str.replace("1 year day max", "Máximo diário anual")
        dataset["metrica"] = dataset.metrica.str.replace("1 year day min", "Mínimo diário anual")
        dataset["tipo_de_estacao"] = dataset.tipo_de_estacao.str.replace("Background", "Fundo")
        dataset["tipo_de_estacao"] = dataset.tipo_de_estacao.str.replace("Traffic", "Tráfego")
        dataset["tipo_de_estacao"] = dataset.tipo_de_estacao.str.replace("Industrial", "Industrial")
        dataset["tipo_de_estacao"] = dataset.tipo_de_estacao.str.replace("Unknown", "Desconhecido")
        dataset["tipo_de_area"] = dataset.tipo_de_area.str.lower()
        dataset["tipo_de_area"] = dataset.tipo_de_area.str.replace("urban", "urbano")
        dataset['poluente_descricao'] = dataset['poluente'].apply(apply_descricao)
        dataset["indice_qualidade_do_ar"] = dataset.apply(lambda x: apply_level(x.poluente, x.nível_de_poluicao), axis=1)
        dataset = dataset.dropna(subset=['nome_estacao'])
        data_.append(dataset)
    final_dataset = pd.concat(data_)
    return final_dataset
                   
                    
                    

In [34]:
dataset = transform_and_translate(data_list).reset_index(drop=True)
dataset.to_csv("qualidade_do_ar.csv")